In [1]:
import json
import boto3
import time
from anthropic import Anthropic 
from botocore.exceptions import ClientError
from chunkipy import TextChunker, TokenEstimator
from transformers import AutoTokenizer #, BertTokenizer, pipeline

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class BertTokenEstimator(TokenEstimator):
    def __init__(self):
        self.bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

    def estimate_tokens(self, text):
        return len(self.bert_tokenizer.encode(text))

In [3]:
# Initialize BertEstimator
bert_token_estimator = BertTokenEstimator()

In [4]:
REGION = "us-east-1"

In [5]:
client_bedrock = boto3.client("bedrock-runtime", REGION)

In [6]:
txt = """Mission Two: The Wrecking Crew
Matt Helm series Reviews by Ujjwal Dey

Well the first one is beaten to grit and grim by this second one in the series. A little longer at 176 pages, still an easy read for my weekend; this one brings Helm into a whole new line of action in espionage business.

Now Matt Helm has had his refresher course in the covert groups training - the American Mordgruppe - The Wrecking Crew - an unknown, unspoken elite group of operatives who generally work alone to do what armies and clouts of bureaucrats can't achieve. The man is just right for the job. The trainers believe he is in no shape to be an operative and certainly past his prime. His bad new resume was certain to get him killed on a field mission. Mac agrees - he is just the man for this job.

Matt Helm now has to play dumb; to act like a clumsy ancient World War trooper who can't call the shots in this peacetime covert warfare. As a photographer for an American magazine he lands up in Artic Europe to shoot innocent bland photos of mines for a girl who could be a double-agent or simply a fool in this foolish game. The girl in question has survived a bullet meant for her journalist husband who had the gall to write a tale describing a Russian agent - The Man No one Knows. Of course this deadly Russian operative has no sympathy for such breakthrough journalism, and now the supposed widow is carrying out her husband's journalistic inclinations.

There is more than meets the eye and pretty women are lethal in more ways than one. Helm's contact in Sweden is shot dead in the face, double-crossed by her evil mysterious agent. Helm has to contend with getting bruised and bashed around to prove himself harmless to a variety of operatives - biding his time to get his orders.

Yes, the men in Washington call off lethal action - no Government ordered assassination during peacetime - but that is not a restriction upon Helm's enemies. Matt Helm goes through unraveling intricacies in the players' cards, as a poker player who has to display ignorance of any known card game. As people show up dead around him, things come to light and when the final game is afoot - Matt has the aces up his sleeve to vindicate Mac's faith in him.

The violence is as bad if not worse than the first book - which is a good thing in any gritty espionage thriller. The brief reflections on the first book events such as him carving up an old lady friend and his separation from his wife also come up very much accurately into the new plot. His handiness without a gun is seen very well in this story as essentially he has to go out there unarmed to convincingly play the role of an American photographer, even if the cover doesn't fool his targets.

The geographical descriptions and accuracy in detail is wonderful and you can imagine yourself tracing his trail across the mountains and into wilderness in the Arctic. He has to "make the touch" - Group M speak for killing the target - similar to what mafia would say "making a hit". But he has to be patient enough to identify the mysterious Russian spy, wait for the go ahead from his Boss, and then make sure he does it cleanly - being in a friendly country during peacetime.

Putting up a classy display of ineptness, we also get to read about all that he could have done as a master agent but doesn't to keep himself useful to the Russian agent - he is able to prove himself harmless on more than one occasion until finally its time for a showdown.

Cars, guns, women's choice of clothes, all again feature in this sequel in Matt Helm's ponderings. There are women he trusts and they assuredly betray him and Helm is not one to be heartbroken or sentimental - he goes about his business with determination and calculation - even surprising his own Government's other operatives (of other departments) - who fall for his "clumsy" act. At the end Helm proves himself to be as cunning and ruthless as his Russian rival. The climax action with its cold-blooded moves sees Helm make his touch and save a damsel from distress as well. The last chapter adds more to Helm's personality and legend. His un-emotive demeanour at what could have been a tragic romantic scene ensures he is the man with a job he is good at.

If you thought gadgets and expensive machinery with latest guns was the way an agent wins a war - you have watched too many James Bond movies. This book's account shows us in a believable and clinical clarity how a secret agent would go through with his mission in a foreign country. Matt Helm is no great fist-fighter but he knows how to fight and here we see him use more of the matter between his ears in contrast to the trigger in enemy hands.

Extract: When you act like a nice guy, everyone examines your motives with a microscope. When you act like a conscienceless louse, they generally take you at face value."""

In [7]:
token_count = bert_token_estimator.estimate_tokens(txt)
print(token_count)

Token indices sequence length is longer than the specified maximum sequence length for this model (1040 > 512). Running this sequence through the model will result in indexing errors


1040


In [ ]:
#def getBedrockResponse(model_id, body):
#    response = client_bedrock.invoke_model(modelId=model_id, body=body)
#    return response

In [8]:
def getBodyJson(system_prompt, message_json):
    body_summary = json.dumps({
        "max_tokens": 2048,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": [ message_json ],
        "anthropic_version": "bedrock-2023-05-31"
    })

    return body_summary

In [9]:
def getMessageJson(user_prompt):
    user_prompt = f"""\n\n\nHuman: {user_prompt}\n\nAssistant:"""
    message_json = {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": user_prompt
            }
        ]
    }
    return message_json

In [10]:
# Define prompt
system_prompt = "The texts here are all book reviews."
user_prompt_1 = f"""Write a summary of the following review; all your output has to be grammatically correct in a single paragraph:\n{txt}"""
user_prompt_2 = f"""The text below is a book review. Tell me the sentiment of the text here:\n{txt}\n The sentiment categories are: \n(1) Positive\n(2) Negative\n(3) Neutral"""
user_prompt_3 = f"""The text below is a book review.\nTell me the sentiment of the text here by returning only one label, which can be (1) Positive, (2) Negative, or (3) Neutral.\nThe text is here:\n{txt}\n
"""
#prompt = f"# Define model ID and prompt
model_id = 'anthropic.claude-v2:1'""System: {system_prompt}\n\nHuman: {user_prompt}\n\nAssistant:"""

message_json_summary = getMessageJson(user_prompt_1)
message_json_sentiment = getMessageJson(user_prompt_2)
message_json_sentiment_label = getMessageJson(user_prompt_3)

body_summary = getBodyJson(system_prompt, message_json_summary)
body_sentiment = getBodyJson(system_prompt, message_json_sentiment)
body_sentiment_label = getBodyJson(system_prompt, message_json_sentiment_label)


In [11]:
# Define model ID
model_id = 'anthropic.claude-v2:1'

In [12]:
# Get start time
t0 = time.localtime()
start_time = time.strftime("%H:%M:%S", t0)
print(start_time)

# Get Anthropic response
response_summary = client_bedrock.invoke_model(modelId=model_id, body=body_summary)

# Get end time
t1 = time.localtime()
end_time = time.strftime("%H:%M:%S", t1)
print(end_time)

21:35:24
21:35:36


In [13]:
# Difference 
diff_time = ( time.mktime(t1) - time.mktime(t0) )
print(f"Difference (seconds): {diff_time}")

Difference (seconds): 12.0


In [14]:
# Get start time
t0 = time.localtime()
start_time = time.strftime("%H:%M:%S", t0)
print(start_time)

# Get Anthropic response
response_sentiment = client_bedrock.invoke_model(modelId=model_id, body=body_sentiment)

# Get end time
t1 = time.localtime()
end_time = time.strftime("%H:%M:%S", t1)
print(end_time)

21:35:39
21:35:53


In [15]:
# Difference 
diff_time = ( time.mktime(t1) - time.mktime(t0) )
print(f"Difference (seconds): {diff_time}")

Difference (seconds): 14.0


In [16]:
# Get start time
t0 = time.localtime()
start_time = time.strftime("%H:%M:%S", t0)
print(start_time)

# Get Anthropic response
response_sentiment_label = client_bedrock.invoke_model(modelId=model_id, body=body_sentiment_label)

# Get end time
t1 = time.localtime()
end_time = time.strftime("%H:%M:%S", t1)
print(end_time)

21:39:37
21:39:46


In [17]:
# Difference 
diff_time = ( time.mktime(t1) - time.mktime(t0) )
print(f"Difference (seconds): {diff_time}")

Difference (seconds): 9.0


# Response

In [18]:
response_body_summary = json.loads(response_summary["body"].read())

In [19]:
print(response_body_summary)

{'id': 'compl_017kf7c3cjgShP83dMGg4x7a', 'type': 'message', 'role': 'assistant', 'content': [{'type': 'text', 'text': 'Here is a one paragraph summary of the book review:\n\nThis is a positive review of the second book in the Matt Helm espionage thriller series, "Mission Two: The Wrecking Crew", which the reviewer says is even more violent than the first book. Helm is sent on a mission to the Arctic as a harmless photographer to identify and eliminate a mysterious Russian spy. He convincingly plays dumb, allowing himself to be betrayed and beaten to prove he is not a threat, while secretly unravelling the complex web of operatives. The plot intricacies and action climax showcase Helm’s ruthless cunning and skill. The geographical detail creates an immersive backdrop as Helm waits patiently for the order to “make the touch” on his target and save a woman in distress, cementing his legend as a coldly competent operative who outsmarts enemies using his intellect rather than weapons.'}], '

In [20]:
print(response_body_summary["content"][0]["text"])

Here is a one paragraph summary of the book review:

This is a positive review of the second book in the Matt Helm espionage thriller series, "Mission Two: The Wrecking Crew", which the reviewer says is even more violent than the first book. Helm is sent on a mission to the Arctic as a harmless photographer to identify and eliminate a mysterious Russian spy. He convincingly plays dumb, allowing himself to be betrayed and beaten to prove he is not a threat, while secretly unravelling the complex web of operatives. The plot intricacies and action climax showcase Helm’s ruthless cunning and skill. The geographical detail creates an immersive backdrop as Helm waits patiently for the order to “make the touch” on his target and save a woman in distress, cementing his legend as a coldly competent operative who outsmarts enemies using his intellect rather than weapons.


In [21]:
response_body_sentiment = json.loads(response_sentiment["body"].read())

In [22]:
print(response_body_sentiment)

{'id': 'compl_01DbAc3Pz8zvUBRRwRquxX4A', 'type': 'message', 'role': 'assistant', 'content': [{'type': 'text', 'text': 'Based on the review, I would characterize the overall sentiment as positive. Here are the main reasons:\n\n1) The review praises the book as better than the first one in the series. Phrases like "this one brings Helm into a whole new line of action" and "a good thing in any gritty espionage thriller" indicate the reviewer enjoyed the second book more.\n\n2) The review highlights several positive aspects - the accuracy of details, the complexity of the plot and characters, the excitement of the climax. Phrases like "wonderful and you can imagine yourself tracing his trail", "unraveling intricacies", and "puts up a classy display" suggest admiration of the author\'s skills.\n\n3) The review favors comparisons to James Bond movies, indicating the reviewer finds the realism and ruthlessness of the main character appealing. Lines like "if you thought gadgets and expensive m

In [23]:
print(response_body_sentiment["content"][0]["text"])

Based on the review, I would characterize the overall sentiment as positive. Here are the main reasons:

1) The review praises the book as better than the first one in the series. Phrases like "this one brings Helm into a whole new line of action" and "a good thing in any gritty espionage thriller" indicate the reviewer enjoyed the second book more.

2) The review highlights several positive aspects - the accuracy of details, the complexity of the plot and characters, the excitement of the climax. Phrases like "wonderful and you can imagine yourself tracing his trail", "unraveling intricacies", and "puts up a classy display" suggest admiration of the author's skills.

3) The review favors comparisons to James Bond movies, indicating the reviewer finds the realism and ruthlessness of the main character appealing. Lines like "if you thought gadgets and expensive machinery with latest guns was the way an agent wins a war - you have watched too many James Bond movies" reinforce this.

Whil

In [24]:
response_body_sentiment_label = json.loads(response_sentiment_label["body"].read())

In [25]:
print(response_body_sentiment_label)

{'id': 'compl_01XwzKmCYHLbuizRE6uF4K4y', 'type': 'message', 'role': 'assistant', 'content': [{'type': 'text', 'text': 'Based on the content and tone of the review, I would categorize this as a Positive sentiment. The review praises various aspects of the book such as the plot, characters, settings, and the author\'s writing style. Some key positive phrases include:\n\n"Well the first one is beaten to grit and grim by this second one in the series."\n"This one brings Helm into a whole new line of action in espionage business."\n"The man is just right for the job."  \n"The violence is as bad if not worse than the first book - which is a good thing in any gritty espionage thriller."\n"The geographical descriptions and accuracy in detail is wonderful and you can imagine yourself tracing his trail across the mountains and into wilderness in the Arctic."\n"Putting up a classy display of ineptness, we also get to read about all that he could have done as a master agent but doesn\'t to keep hi

In [26]:
print(response_body_sentiment_label["content"][0]["text"])

Based on the content and tone of the review, I would categorize this as a Positive sentiment. The review praises various aspects of the book such as the plot, characters, settings, and the author's writing style. Some key positive phrases include:

"Well the first one is beaten to grit and grim by this second one in the series."
"This one brings Helm into a whole new line of action in espionage business."
"The man is just right for the job."  
"The violence is as bad if not worse than the first book - which is a good thing in any gritty espionage thriller."
"The geographical descriptions and accuracy in detail is wonderful and you can imagine yourself tracing his trail across the mountains and into wilderness in the Arctic."
"Putting up a classy display of ineptness, we also get to read about all that he could have done as a master agent but doesn't to keep himself useful to the Russian agent"

There are no overtly negative sentiments expressed. Therefore, I conclude this is a Positive

# Using Claude 3 Haiku

In [27]:
# Define model ID and prompt
model_id = 'anthropic.claude-3-haiku-20240307-v1:0'


In [28]:
# Get start time
t0 = time.localtime()
start_time = time.strftime("%H:%M:%S", t0)
print(start_time)

# Get Anthropic response
response_summary = client_bedrock.invoke_model(modelId=model_id, body=body_summary)

# Get end time
t1 = time.localtime()
end_time = time.strftime("%H:%M:%S", t1)
print(end_time)

21:40:15
21:40:18


In [29]:
# Difference 
diff_time = ( time.mktime(t1) - time.mktime(t0) )
print(f"Difference (seconds): {diff_time}")

Difference (seconds): 3.0


In [30]:
response_body = json.loads(response_summary["body"].read())

In [31]:
print(response_body)

{'id': 'msg_01Eack1JVTXWzunERZWVowz8', 'type': 'message', 'role': 'assistant', 'content': [{'type': 'text', 'text': 'The review discusses the second book in the Matt Helm series, "The Wrecking Crew." It highlights how this installment brings Helm into a new realm of espionage, where he must play the role of a clumsy, aging World War II veteran photographer to infiltrate a mission in Arctic Europe. The review praises the book\'s gritty and violent nature, as well as the accurate geographical descriptions and Helm\'s calculated approach to his mission, which involves outsmarting his targets and proving himself harmless until the time is right for him to strike. The review also notes Helm\'s ability to surprise even his own government\'s operatives with his cunning and ruthlessness, and the book\'s emphasis on the mental aspect of espionage rather than relying on gadgets or brute force. Overall, the review suggests that this sequel in the Matt Helm series is a compelling and realistic por

In [32]:
print(response_body["content"][0]["text"])

The review discusses the second book in the Matt Helm series, "The Wrecking Crew." It highlights how this installment brings Helm into a new realm of espionage, where he must play the role of a clumsy, aging World War II veteran photographer to infiltrate a mission in Arctic Europe. The review praises the book's gritty and violent nature, as well as the accurate geographical descriptions and Helm's calculated approach to his mission, which involves outsmarting his targets and proving himself harmless until the time is right for him to strike. The review also notes Helm's ability to surprise even his own government's operatives with his cunning and ruthlessness, and the book's emphasis on the mental aspect of espionage rather than relying on gadgets or brute force. Overall, the review suggests that this sequel in the Matt Helm series is a compelling and realistic portrayal of a secret agent's work in a foreign country during peacetime.


In [33]:
# Get start time
t0 = time.localtime()
start_time = time.strftime("%H:%M:%S", t0)
print(start_time)

# Get Anthropic response
response_sentiment = client_bedrock.invoke_model(modelId=model_id, body=body_sentiment)

# Get end time
t1 = time.localtime()
end_time = time.strftime("%H:%M:%S", t1)
print(end_time)

21:40:23
21:40:26


In [34]:
# Difference 
diff_time = ( time.mktime(t1) - time.mktime(t0) )
print(f"Difference (seconds): {diff_time}")

Difference (seconds): 3.0


In [35]:
response_body = json.loads(response_sentiment["body"].read())

In [36]:
print(response_body)

{'id': 'msg_013JFNpgrvsAs4rqxv3wzZrF', 'type': 'message', 'role': 'assistant', 'content': [{'type': 'text', 'text': 'The sentiment of the book review is predominantly positive. The reviewer seems to appreciate the depth and complexity of the Matt Helm character, as well as the realistic and gritty nature of the espionage action depicted in the book. Some key points that indicate a positive sentiment:\n\n1. The reviewer states that the violence in this book is "as bad if not worse than the first book - which is a good thing in any gritty espionage thriller."\n\n2. The reviewer praises the geographical descriptions and accuracy in detail, saying "you can imagine yourself tracing his trail across the mountains and into wilderness in the Arctic."\n\n3. The reviewer appreciates how the book shows "in a believable and clinical clarity how a secret agent would go through with his mission in a foreign country."\n\n4. The reviewer highlights positive attributes of the main character, Matt Helm,

In [37]:
print(response_body["content"][0]["text"])

The sentiment of the book review is predominantly positive. The reviewer seems to appreciate the depth and complexity of the Matt Helm character, as well as the realistic and gritty nature of the espionage action depicted in the book. Some key points that indicate a positive sentiment:

1. The reviewer states that the violence in this book is "as bad if not worse than the first book - which is a good thing in any gritty espionage thriller."

2. The reviewer praises the geographical descriptions and accuracy in detail, saying "you can imagine yourself tracing his trail across the mountains and into wilderness in the Arctic."

3. The reviewer appreciates how the book shows "in a believable and clinical clarity how a secret agent would go through with his mission in a foreign country."

4. The reviewer highlights positive attributes of the main character, Matt Helm, such as his cunning, ruthlessness, and ability to use his mind rather than just rely on gadgets and firearms.

Overall, the 

In [38]:
# Get start time
t0 = time.localtime()
start_time = time.strftime("%H:%M:%S", t0)
print(start_time)

# Get Anthropic response
response_sentiment_label = client_bedrock.invoke_model(modelId=model_id, body=body_sentiment_label)

# Get end time
t1 = time.localtime()
end_time = time.strftime("%H:%M:%S", t1)
print(end_time)

21:40:46
21:40:47


In [39]:
# Difference 
diff_time = ( time.mktime(t1) - time.mktime(t0) )
print(f"Difference (seconds): {diff_time}")

Difference (seconds): 1.0


In [40]:
response_body = json.loads(response_sentiment_label["body"].read())

In [41]:
print(response_body)

{'id': 'msg_015pshHVET17UHDYFnGZm3L5', 'type': 'message', 'role': 'assistant', 'content': [{'type': 'text', 'text': 'The sentiment of the text is Positive.'}], 'model': 'claude-3-haiku-48k-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 1190, 'output_tokens': 12}}


In [42]:
print(response_body["content"][0]["text"])

The sentiment of the text is Positive.
